In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')[::10]
x_test = x_test.astype('float32')[::10]
y_train = y_train[::10]
y_test = y_test[::10]
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], y_train.shape, 'train samples')
print(x_test.shape[0], y_test.shape, 'test samples')

Using TensorFlow backend.


x_train shape: (6000, 28, 28, 1)
6000 (6000,) train samples
1000 (1000,) test samples


In [2]:
def triplet_loss(X):
    anchor, pos, neg = X
    return K.sum(K.pow(anchor - pos, 2) - K.pow(anchor - neg, 2))

def identity_loss(y_true, y_pred):
    return K.mean(y_pred - 0 * y_true)

In [30]:
from keras.models import Model
from keras.layers import Input, Embedding, merge
from keras.optimizers import Adam

embedding_length = 4

digit_input = Input(shape=input_shape)
x = Conv2D(64, (3, 3))(digit_input)
x = Conv2D(64, (3, 3))(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
out = Dense(embedding_length, activation="sigmoid")(x)

vision_model = Model(digit_input, out)

anchor = Input(input_shape)
anchor_out = vision_model(anchor)

pos = Input(input_shape)
pos_out = vision_model(pos)

neg = Input(input_shape)
neg_out = vision_model(neg)

loss = merge(
    [anchor_out, pos_out, neg_out],
    mode=triplet_loss,
    name='loss',
    output_shape=(1, )
)

model = Model(inputs=[anchor, pos, neg], outputs=[loss])
model.compile(loss=identity_loss, optimizer=Adam(lr=1e-4))

/home/joe/upwork/.env/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/joe/upwork/.env/lib/python3.5/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [31]:
import numpy as np
def generate_triplets():
    i = 0
    while True:        
        i += 1
        idx = np.random.randint(0, y_train.shape[0])
        anchor, cls = x_train[[idx]], y_train[[idx]]
        pos = x_train[y_train==cls][[i%(y_train==cls).sum()]]
        neg = x_train[y_train!=cls][[i%(y_train==cls).sum()]]
        yield ([anchor, pos, neg], cls)

In [32]:
model.fit_generator(generate_triplets(), steps_per_epoch=512, epochs=4)

Epoch 1/4
512/512 [==============================] - 14s - loss: -0.6338    
Epoch 2/4
512/512 [==============================] - 16s - loss: -1.2328    
Epoch 3/4
512/512 [==============================] - 16s - loss: -1.2601    
Epoch 4/4
512/512 [==============================] - 16s - loss: -1.4071    


In [33]:
test_model = Model(inputs=[anchor], outputs=[anchor_out])

In [37]:
for i in range(10):
    print(test_model.predict(x_test[[i]]).round(1), y_test[i])

[[ 1.  1.  0.  1.]] 7
[[ 0.  0.  1.  0.]] 0
[[ 1.  1.  0.  1.]] 9
[[ 0.          0.          0.89999998  0.1       ]] 3
[[ 1.  0.  0.  1.]] 1
[[ 0.  1.  1.  0.]] 6
[[ 1.  1.  0.  1.]] 7
[[ 1.  1.  0.  1.]] 7
[[ 1.  1.  0.  1.]] 7
[[ 0.  0.  1.  0.]] 3
